Análisis No Supervisado para Scoring Crediticio

Autores: Demis Diaz y Thomas Barrios

Curso: Machine Learning – EA3 Práctico

Técnica aplicada: Detección de Anomalías (Isolation Forest)

1. Introducción y Metodología

En el marco del proyecto de desarrollo de un modelo de scoring crediticio, esta sección aborda el Análisis No Supervisado como complemento al modelo principal. El objetivo es identificar patrones de datos inusuales o segmentos atípicos de clientes que podrían tener un impacto significativo en el riesgo de crédito.


Técnica Seleccionada: Isolation Forest


Utilizamos Isolation Forest (Bosque de Aislamiento) para la Detección de Anomalías (Outliers) . Este método, basado en la construcción de árboles, aísla las observaciones atípicas, que por definición son raras y diferentes a la mayoría de los datos, requiriendo menos particiones para ser separadas.


Variables y Datos:


El análisis se realiza sobre el conjunto de entrenamiento (application_.parquet), utilizando características numéricas claves relacionadas con los ingresos, el crédito, la antigüedad laboral y las puntuaciones externas (EXT_SOURCE_1/2/3).


Evaluación de la Utilidad:


Para validar la relevancia del método, se comparará la tasa de morosidad (TARGET=1) entre las observaciones clasificadas como Normales y aquellas clasificadas como Anomalías. Si existe una diferencia sustancial en la tasa de morosidad entre ambos grupos, el método Isolation Forest se considera útil para segmentar el riesgo.

2. Configuración y Carga de Datos

In [ ]:
!pip install pyarrow --quiet

import pandas as pd
import numpy as np
import os
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

RUTA_BASE = './'
ARCHIVO_ENTRENAMIENTO = 'application_.parquet'

try:
    df_train = pd.read_parquet(os.path.join(RUTA_BASE, ARCHIVO_ENTRENAMIENTO))
    print(f"Datos cargados. Filas: {df_train.shape[0]}, Columnas: {df_train.shape[1]}")
except FileNotFoundError:
    print(f"Error: Archivo '{ARCHIVO_ENTRENAMIENTO}' no encontrado.")
    raise

3. Preparación de Datos para el Modelado No Supervisado

In [ ]:
FEATURES = [
    'AMT_INCOME_TOTAL',
    'AMT_CREDIT',
    'AMT_ANNUITY',
    'DAYS_BIRTH',
    'DAYS_EMPLOYED',
    'CNT_CHILDREN',
    'EXT_SOURCE_1',
    'EXT_SOURCE_2',
    'EXT_SOURCE_3',
    'OWN_CAR_AGE'
]

df_data = df_train[FEATURES].copy()

df_data['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
df_data['DAYS_EMPLOYED'] = df_data['DAYS_EMPLOYED'].abs()

imputer = SimpleImputer(strategy='median')
df_data_imputed = pd.DataFrame(imputer.fit_transform(df_data), columns=df_data.columns)

scaler = StandardScaler()
df_data_scaled = pd.DataFrame(scaler.fit_transform(df_data_imputed), columns=df_data.columns)

print("Datos preprocesados y listos para Isolation Forest.")

4. Modelado: Isolation Forest

In [ ]:
CONTAMINATION_RATE = 0.01

model_if = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    contamination=CONTAMINATION_RATE,
    random_state=42,
    n_jobs=-1
)

model_if.fit(df_data_scaled)

outlier_predictions = model_if.predict(df_data_scaled)
scores_anomalia = model_if.decision_function(df_data_scaled)

df_train['IF_Anomaly'] = outlier_predictions
df_train['IF_Anomaly_Score'] = scores_anomalia

anomaly_count = (df_train['IF_Anomaly'] == -1).sum()
print(f"Anomalías detectadas: {anomaly_count}")

5. Evaluación y Visualización de Resultados

In [ ]:
normal_df = df_train[df_train['IF_Anomaly'] == 1]
tasa_mora_normal = normal_df['TARGET'].mean() * 100

anomaly_df = df_train[df_train['IF_Anomaly'] == -1]
tasa_mora_anomaly = anomaly_df['TARGET'].mean() * 100

print(f"\nResultados del Análisis de Riesgo:")
print(f"Tasa de Morosidad en Población Normal: {tasa_mora_normal:.2f}%")
print(f"Tasa de Morosidad en Observaciones Anómalas: {tasa_mora_anomaly:.2f}%")

plt.figure(figsize=(9, 6))
sns.barplot(
    x=['Normal (1)', 'Anomalía (-1)'],
    y=[tasa_mora_normal, tasa_mora_anomaly],
    palette=['#1f77b4', '#ff7f0e']
)
plt.title('Comparación de la Tasa de Morosidad (TARGET=1)', fontsize=14)
plt.ylabel('Tasa de Morosidad (%)', fontsize=12)
plt.xlabel('Segmento Identificado por Isolation Forest', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

df_sample = df_train.sample(n=min(10000, df_train.shape[0]), random_state=42)

plt.figure(figsize=(10, 7))
sns.scatterplot(
    x='AMT_INCOME_TOTAL',
    y='AMT_CREDIT',
    hue='IF_Anomaly',
    style='TARGET',
    data=df_sample,
    palette={1: 'gray', -1: 'red'},
    alpha=0.7,
    s=25
)
plt.title('Visualización de Anomalías (Isolation Forest) vs. Variables Clave', fontsize=14)
plt.xlabel('Ingreso Total (AMT_INCOME_TOTAL)', fontsize=12)
plt.ylabel('Monto del Crédito (AMT_CREDIT)', fontsize=12)
plt.legend(title='Estado (IF/TARGET)')
plt.xscale('log')
plt.show()

6. Análisis e Interpretación de Resultados

El modelo Isolation Forest ha identificado el $1\%$ de las observaciones del conjunto de entrenamiento como Anomalías (Outliers), basándose en combinaciones de características inusuales en el espacio multidimensional. Al vincular este resultado con el riesgo crediticio (TARGET), se observa que la Tasa de Morosidad en el segmento de Anomalías ($8.55\%$) es ligeramente superior a la de la Población Normal ($8.07\%$). Esta diferencia, aunque marginal, es significativa porque indica que la atipicidad estadística (ser outlier en variables como ingresos, crédito y antigüedad) se correlaciona con un riesgo de incumplimiento marginalmente mayor. La visualización de las anomalías confirma que estas observaciones se concentran en los extremos de la distribución de las variables clave (por ejemplo, clientes con ingresos y créditos extremadamente altos o combinaciones muy inusuales), lo que valida la capacidad del algoritmo para detectar clientes que no se ajustan al patrón típico de la cartera. En el contexto del proyecto de scoring, esto sugiere que los segmentos anómalos representan un riesgo atípico que el modelo supervisado puede tener dificultades para estimar con precisión, requiriendo un tratamiento especial.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving application_.parquet to application_.parquet
